## Step 3: Review Understanding Agent

This agent converts raw Swiggy reviews into structured topic signals
using high-recall semantic rules.


In [2]:
TOPIC_RULES = {
    "delivery partner rude": ["rude", "impolite", "bad behavior", "misbehaved"],
    "delivery delay": ["late", "delay", "delayed"],
    "food quality issue": ["stale", "cold", "bad food", "spoiled", "worst food"],
    "customer support issue": ["customer service", "support", "help", "cannot help"],
    "refund issue": ["refund", "money not returned", "refund not received"],
    "instamart availability": ["instamart", "out of stock", "closed"],
    "app issue": ["app crash", "not working", "bug", "map", "tracking"]
}


In [4]:
def extract_topic_local(review):
    review = review.lower()

    for topic, keywords in TOPIC_RULES.items():
        for kw in keywords:
            if kw in review:
                return topic

    positive_words = ["good", "nice", "best", "great", "awesome", "excellent", "super"]
    if review.strip() in positive_words or any(pw in review for pw in positive_words):
        return "positive feedback"

    return "general feedback"


In [6]:
import pandas as pd

df = pd.read_csv("../data/swiggy_reviews_raw.csv")
df["review_date"] = pd.to_datetime(df["review_date"]).dt.date

daily_batches = {
    date: group["review_text"].tolist()
    for date, group in df.groupby("review_date")
}

all_rows = []

for date, reviews in daily_batches.items():
    for r in reviews:
        all_rows.append({
            "review_date": date,
            "review_text": r,
            "topic_candidate": extract_topic_local(r)
        })

understanding_df = pd.DataFrame(all_rows)
understanding_df.head()


,review_date,review_text,topic_candidate
0,2025-12-23,Excellent service 💯,positive feedback
1,2025-12-23,super,positive feedback
2,2025-12-23,worst,general feedback
3,2025-12-23,❤️,general feedback
4,2025-12-23,good,positive feedback
